# Preperation about the conda environment
Create a conda environment called "manipulator" with python 3.10 and pip3 installed:    
+ 🚀 `conda create -n manipulator python=3.10 `   

Activate the environment "manipulator" and start installation:   
+ 🚀 `conda activate manipulator`  

Install pytorch according to the tutorial of [Pytorch Official Website](https://pytorch.org/):   
+ 🚀 `conda install pytorch torchvision torchaudio pytorch-cuda=<your_version> -c pytorch -c nvidia`   

Install package for juypter notebook:  
+ 🚀 `conda install ipykernel` *# juypter notebook use for VScode*
+ 🚀 `conda install jupyter notebook`  

Pip install packages for further use:   
+ 🚀 `pip install matplotlib` *# display the results*
+ 🚀 `pip install pandas` *# pandas for debugging and matrices*
+ 🚀 `pip install scipy`  *# physical parameters*

In [1]:
# Module Import
import torch
import json
import numpy as np

In [2]:
# GPU check
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
print(f'Pytorch version: {torch.__version__}')
print(f'CUDA version: {torch.version.cuda}')

Using device: cuda
Pytorch version: 2.1.2
CUDA version: 12.1


# Change the angle over there

In [3]:
length = 300
Sr = 0.5 * length
d = 0.015 * length
angle = torch.tensor([90, 90, -90, -90])
radians = torch.deg2rad(angle) # convert degrees to radians

In [4]:
# Dictionary Initialization
M = [{'bend': torch.eye(3), 'bend_inv': torch.eye(3), 'displacement': torch.zeros(3, 1), 
      'B': torch.eye(3)} for _ in range(5)]
position = torch.zeros(3, 5)
node = [{} for _ in range(5)]

# Orientation Calculation
for i in range(0, 4):
    alpha = radians[i]
    if torch.sign(alpha) == 0:
        Dhorz = 0
        Dvert = Sr + d
    else:
        R = Sr / alpha
        Dhorz = R * (1 - np.cos(alpha)) + d * np.sin(alpha)
        Dvert = R * np.sin(alpha) + d * np.cos(alpha)
    if i % 2 == 0:
        Mt = torch.tensor([[1, 0, 0],
            [0, torch.cos(alpha), torch.sin(alpha)],
            [0, -torch.sin(alpha), torch.cos(alpha)]])
        Mt_inv = torch.tensor([[1, 0, 0],
            [0, torch.cos(alpha), -torch.sin(alpha)],
            [0, torch.sin(alpha), torch.cos(alpha)]])
        M[i]['displacement'] = torch.tensor([[0],[Dhorz],[Dvert]])
    else:
        Mt = torch.tensor([[torch.cos(alpha), 0, torch.sin(alpha)],
                                    [0, 1, 0],
                                    [-torch.sin(alpha), 0, torch.cos(alpha)]])

        Mt_inv = torch.tensor([[torch.cos(alpha), 0, -torch.sin(alpha)],
                                        [0, 1, 0],
                                        [torch.sin(alpha), 0, torch.cos(alpha)]])

        M[i]['displacement'] = torch.tensor([[Dhorz],[0],[Dvert]])
    M[i+1]['bend'] = torch.round(Mt * 1e5) / 1e5
    M[i+1]['bend_inv'] = torch.round(Mt_inv * M[i]['bend_inv'] * 1e5) / 1e5

for i in range(len(M)):
    for j in range(i + 1):
        M[i]['B'] = torch.mm(M[i]['B'], M[j]['bend'])

# Set the original point
node[0]['position'] = position[:, 0]

# Node position calculation
for i in range(1, 5):
    position[:, i] = torch.mm(M[i-1]['B'], M[i]['displacement']).squeeze() + position[:, i-1]
    node[i]['position'] = position[:, i]

[print(f'Node position for element {i+1}:\n{node[i]["position"]}\n') for i in range(5)]

position_np = position.numpy()
with open ('data/data.json', 'a') as json_file:
    json.dump(position_np.tolist(), json_file)
    json_file.write('\n')

Node position for element 1:
tensor([0., 0., 0.])

Node position for element 2:
tensor([99.9930,  0.0000, 95.4930])

Node position for element 3:
tensor([ 99.9930,  95.4930, 195.4859])

Node position for element 4:
tensor([195.4859, 195.4859, 195.4859])

Node position for element 5:
tensor([195.4859, 195.4859, 195.4859])

